# AIM
In this notebook we show the prediction accuracy for deep CNNs such as
- MobileNetV2
- VGG19
- ResNet 50V2
- Xception


plus two other simpler models one tuned and one similar to LeNet5. 
Also we export the models to ONNX. 

## Setting Paths

In [13]:
import sys
from pathlib import Path
#Set root to be the main project folder
#Note that this notebook is in /SOCIAL_DISTANCING/CODE/NOTEBOOKS/TEST_NOTEBOOKS
root = Path.cwd().parent
print(f"{root=}")

root=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma')


In [14]:
data_path = Path(root/'data')
print(f"{data_path=}")
sav_models = Path(root/'saved_models')
print(f"{sav_models =}")

#Add location of py files to path so we can import
sys.path.insert(0,str(root))

data_path=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/data')
sav_models =PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/saved_models')


## LIBRARIES / PACKAGES

In [15]:
import os
import json
import tensorflow as tf
import keras2onnx
import tf2onnx
import onnx

## Importing Models

In [16]:
from models.deepnet import DeepNet
from models.lenet import LeNet

### Importing Methods from Utils

In [17]:
from utils.tools import ImageGenerator, FlowFromDir

### Checking if GPU has been detected

In [18]:
device_name = tf.test.gpu_device_name()
if device_name == '':
    raise SystemError("GPU not found")
else:
    print(f"GPU {device_name} found")

GPU /device:GPU:0 found


2021-08-06 10:14:57.960308: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 10:14:57.960911: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce GTX 980 computeCapability: 5.2
coreClock: 1.329GHz coreCount: 16 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-06 10:14:57.960988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 10:14:57.961360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 10:14:57.961668: I tensorflow/core/common_runtim

## Processing Images

#### Setting paths

In [19]:
dir_with_zip = data_path
dir_with_data = f"{dir_with_zip}/DermMel" 
path_train = f"{dir_with_data}/train_sep/"
path_val = f"{dir_with_data}/valid/"
path_test = f"{dir_with_data}/test/"

#### Using ImageDataGenerator
See https://keras.io/api/preprocessing/image/

In [20]:
data_train = ImageGenerator()
data_val = ImageGenerator()
data_test = ImageGenerator()

In [21]:
train_generator = FlowFromDir(
    data_train,
    path_train,
    target_size=(160, 160),
    shuffle=True)
val_generator = FlowFromDir(
    data_val,
    path_val,
    target_size=(160, 160),
    shuffle=True)
test_generator = FlowFromDir(
    data_test,
    path_test,
    target_size=(160, 160),
    batch_size=1)

Found 10683 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


In [22]:
train_generator

### Input and output shapes

In [11]:
input_shape = train_generator[0][0].shape[1:]
input_shape

(160, 160, 3)

In [12]:
num_of_classes = train_generator[0][1][0].shape[0]
num_of_classes 

2

# MobileNetV2 - Initiating the model

### Setting model parameters

In [14]:
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)
mobilenetv2_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                    include_top=False,
                                                    weights='imagenet',
                                                    pooling=None, 
                                                    classifier_activation="softmax"
                                                )

In [15]:
MobModel = DeepNet(mobilenetv2_model, num_of_classes)
MobModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

## Prediction accuracy on test dataset

In [16]:
MobModel.call(inputs)
MobModel.built = True 
MobModel.load_weights(sav_models/"weights/mobnet.h5")
MobModel.summary()

Model: "deep_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               153720    
_________________________________________________________________
dense_1 (Dense)              (None, 84)                10164     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 170       
_________________________________________________________________
dropout (Dropout)            multiple                  0 (unused)
Total params: 2,422,038
Trainable params: 2,387,926
Non-trainable params: 34,112
___________________________________________

In [19]:
test_generator.reset()
MobModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 34s 9ms/step - loss: 0.1352 - accuracy: 0.9495


[0.1352062076330185, 0.9494524002075195]

## Exporting to ONNX

In [20]:
onnx_model = keras2onnx.convert_keras(MobModel, MobModel.name)
temp_model_file = sav_models/"onnx/mobnet.onnx"
keras2onnx.save_model(onnx_model, temp_model_file)

# VGG19 - Initiating the model

In [28]:
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)
vgg19_model = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling=None, 
    classifier_activation='softmax'
)

In [29]:
Vgg19Model = DeepNet(vgg19_model, num_of_classes)
Vgg19Model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

### Model Summary

## Prediction accuracy on test dataset

In [31]:
Vgg19Model.call(inputs)
Vgg19Model.built = True 
Vgg19Model.load_weights(sav_models/"weights/vgg19.h5")
Vgg19Model.summary()

Model: "deep_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 5, 5, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 120)               61560     
_________________________________________________________________
dense_10 (Dense)             (None, 84)                10164     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 170       
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0 (unused)
Total params: 20,096,278
Trainable params: 20,096,278
Non-trainable params: 0
____________________________________________

In [32]:
test_generator.reset()
Vgg19Model.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 40s 11ms/step - loss: 0.1638 - accuracy: 0.9405


[0.16375385224819183, 0.9404661655426025]

## Exporting to ONNX

In [49]:
onnx_model = keras2onnx.convert_keras(Vgg19Model, Vgg19Model.name)
temp_model_file = sav_models/"onnx/vgg19.onnx"
keras2onnx.save_model(onnx_model, temp_model_file)

tf executing eager_mode: True
tf.keras model eager_mode: False
2021-08-05 21:31:43.075610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-05 21:31:43.075792: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-08-05 21:31:43.075850: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-08-05 21:31:43.076189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-05 21:31:43.076452: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce GTX 980 computeCapability: 5.2
coreClock: 1.329GHz coreCount: 16 deviceMemorySize: 3.95GiB d

# ResNet 50V2 - Initiating the model

In [33]:
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)
resnet_model = tf.keras.applications.ResNet50V2(
    include_top=False, 
    weights="imagenet",
    input_shape=input_shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [34]:
ResNetModel = DeepNet(resnet_model, num_of_classes)
ResNetModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

## Prediction accuracy on test dataset

In [35]:
ResNetModel.call(inputs)
ResNetModel.built = True 
ResNetModel.load_weights(sav_models/"weights/resnet.h5")
ResNetModel.summary()

Model: "deep_net_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 5, 5, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 120)               245880    
_________________________________________________________________
dense_13 (Dense)             (None, 84)                10164     
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 170       
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0 (unused)
Total params: 23,821,014
Trainable params: 23,775,574
Non-trainable params: 45,440
_______________________________________

In [36]:
test_generator.reset()
ResNetModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 43s 12ms/step - loss: 0.2095 - accuracy: 0.9096


[0.20945307612419128, 0.9095759391784668]

## Exporting to ONNX

In [ ]:
onnx_model = keras2onnx.convert_keras(ResNetModel, ResNetModel.name)
temp_model_file = sav_models/"onnx/resnet.onnx"
keras2onnx.save_model(onnx_model, temp_model_file)

# Xception - Initiating the model

In [37]:
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)
xception_model = tf.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling=None, 
    classifier_activation='softmax'
)

In [38]:
XcpModel = DeepNet(xception_model, num_of_classes)
XcpModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

## Prediction accuracy on test dataset

In [39]:
XcpModel.call(inputs)
XcpModel.built = True 
XcpModel.load_weights(sav_models/"weights/xception.h5")
XcpModel.summary()

Model: "deep_net_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 120)               245880    
_________________________________________________________________
dense_16 (Dense)             (None, 84)                10164     
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 170       
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0 (unused)
Total params: 21,117,694
Trainable params: 21,063,166
Non-trainable params: 54,528
_______________________________________

In [43]:
test_generator.reset()
XcpModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 37s 10ms/step - loss: 0.1274 - accuracy: 0.9497


[0.12744712829589844, 0.9497331976890564]

## Exporting to ONNX

In [ ]:
onnx_model = keras2onnx.convert_keras(XcpModel, XcpModel.name)
temp_model_file = sav_models/"onnx/xception.onnx"
keras2onnx.save_model(onnx_model, temp_model_file)

# Simple CNN (LeNet alike) 

In [12]:
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

In [13]:
BaselineModel = LeNet(num_of_classes)
BaselineModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

2021-08-06 08:05:04.713715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:05:04.714010: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce GTX 980 computeCapability: 5.2
coreClock: 1.329GHz coreCount: 16 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-06 08:05:04.714084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:05:04.714335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:05:04.714589: I tensorflow/core/common_runtim

In [14]:
BaselineModel.call(inputs)
BaselineModel.built = True 
BaselineModel.load_weights(sav_models/"weights/lenet.h5")
BaselineModel.summary()

Model: "le_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 32)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 38, 38, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 92416)             0         
_________________________________________________________________
dense (Dense)                (None, 120)               11090040  
_________________________________________________________________
dense_1 (Dense)              (None, 84)                10164

## Prediction accuracy on test dataset

In [15]:
test_generator.reset()
BaselineModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

2021-08-06 08:05:24.143815: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-06 08:05:24.162529: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3399905000 Hz
2021-08-06 08:05:24.421487: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-06 08:05:24.735020: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-06 08:05:24.988907: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-06 08:05:24.988947: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-08-06 08:05:24.989669: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-06 08:05:24.989735: W tensorflow/stream_executor/gpu/redzone_allocator.cc:31

  17/3561 [..............................] - ETA: 22s - loss: 0.0849 - accuracy: 1.0000   

2021-08-06 08:05:25.712476: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


3561/3561 [==============================] - 34s 9ms/step - loss: 0.2687 - accuracy: 0.8874


[0.26874303817749023, 0.887391209602356]

## Exporting to ONNX

In [16]:
onnx_model = keras2onnx.convert_keras(BaselineModel, BaselineModel.name)
temp_model_file = sav_models/"onnx/lenet.onnx"
keras2onnx.save_model(onnx_model, temp_model_file)

tf executing eager_mode: True
tf.keras model eager_mode: False
2021-08-06 08:06:20.424612: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-08-06 08:06:20.429966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:06:20.430319: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-08-06 08:06:20.431357: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-08-06 08:06:20.431714: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:06:20.431902: I tensorflow/core/common_runtime/gpu/gpu

# Simple CNN (tuned)

In [35]:
json_file = open(sav_models/"json/3ConvTuned.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
TunCNNModel = tf.keras.models.model_from_json(loaded_model_json)

In [36]:
TunCNNModel.call(inputs)
TunCNNModel.built = True 
TunCNNModel.load_weights(sav_models/"weights/3ConvTuned.h5")
TunCNNModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)
TunCNNModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 158, 158, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 128)       36992     
_________________________________________________________________
batch_normalization_1 (Batch (None, 77, 77, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 64)        7

In [24]:
test_generator.reset()
TunCNNModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 31s 9ms/step - loss: 0.2053 - accuracy: 0.9174


[0.2053224891424179, 0.9174389243125916]

## Exporting to ONNX

In [34]:
onnx_model, _ = tf2onnx.convert.from_keras(TunCNNModel)
temp_model_file = sav_models/"onnx/3ConvTuned.onnx"
onnx.save(onnx_model, temp_model_file)